In [1]:
PATH = "results/basecnn200" # this is for augmented
# PATH = "basecnn200_source" # this is for source data

In [26]:
from util import *
import torch

preds, truths = load_pickle(f"{PATH}/predictions.pkl")
model_set = torch.load(f"{PATH}/model.pt")
model = model_set["model"]

In [3]:
from torchsummary import summary


In [8]:
mod = model_set["model"]

In [15]:
from models_cnn import *

mod2 = BaseCNN(num_classes=200)

In [20]:
summary(mod, (17, 216))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 17, 108]             256
         MaxPool2d-2            [-1, 16, 8, 27]               0
            Conv2d-3            [-1, 32, 8, 25]           7,712
         MaxPool2d-4             [-1, 32, 4, 6]               0
           Dropout-5                  [-1, 768]               0
            Linear-6                  [-1, 128]          98,432
           Dropout-7                  [-1, 128]               0
            Linear-8                  [-1, 200]          25,800
Total params: 132,200
Trainable params: 132,200
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.31
Params size (MB): 0.50
Estimated Total Size (MB): 0.83
----------------------------------------------------------------


In [25]:
import torchinfo

torchinfo.summary(mod, input_size=(1, 17, 216))

Layer (type:depth-idx)                   Output Shape              Param #
BaseCNN                                  [1, 200]                  --
├─Conv2d: 1-1                            [1, 16, 17, 108]          256
├─MaxPool2d: 1-2                         [1, 16, 8, 27]            --
├─Conv2d: 1-3                            [1, 32, 8, 25]            7,712
├─MaxPool2d: 1-4                         [1, 32, 4, 6]             --
├─Dropout: 1-5                           [1, 768]                  --
├─Linear: 1-6                            [1, 128]                  98,432
├─Dropout: 1-7                           [1, 128]                  --
├─Linear: 1-8                            [1, 200]                  25,800
Total params: 132,200
Trainable params: 132,200
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.14
Input size (MB): 0.01
Forward/backward pass size (MB): 0.29
Params size (MB): 0.53
Estimated Total Size (MB): 0.83

In [23]:
!pip install torchviz

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.2 MB/s eta 0:00:00
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4133 sha256=15c369e7318bcdc0b4081dea166a777d0c455fa536551b43cab6274ffbb9e31d
  Stored in directory: /Users/m243289/Library/Caches/pip/wheels/5a/d0/3f/b7014553eb74f12892b7d9b69c6083044564712d10fde8dfdc
Successfully built torchviz

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [24]:
from torchviz import make_dot

x = torch.randn(1, 17, 216, requires_grad=True)
y = mod2(x)

make_dot(y, params=dict(list(mod2.named_parameters())))

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [27]:

pred_train, prob_train, true_train = predict(model, model_set["train_loader"], get_cuda_device())
pred_val, prob_val, true_val = predict(model, model_set["val_loader"], get_cuda_device())
# concatenate the predictions as they are 18 arrays
pred_train = np.concatenate(pred_train, axis=0)
true_train = np.concatenate(true_train, axis=0)

prob_train = np.concatenate(prob_train, axis=0)
prob_val = np.concatenate(prob_val, axis=0)

pred_val = np.concatenate(pred_val, axis=0)
true_val = np.concatenate(true_val, axis=0)


194it [00:11, 16.94it/s]
30it [00:02, 13.82it/s]


In [28]:
print(true_train.shape)
print(true_val.shape)
print(len(set(true_train)))

print(pred_train.shape)
print(pred_val.shape)
print(len(set(pred_train)))

print(prob_train.shape)
print(prob_val.shape)

(49664,)
(7450,)
200
(49664,)
(7450,)
200
(49664, 200)
(7450, 200)


In [29]:
# subset prob_val to only classes in true_val
prob_val = prob_val[:, list(set(true_val))]

In [31]:

from sklearn.metrics import roc_auc_score

auc_train = roc_auc_score(true_train, prob_train, average='macro', multi_class='ovr')
auc_val = roc_auc_score(true_val, prob_val, average='macro', multi_class='ovr')
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

# create a table for training and validation metrics
metrics = {
    "Accuracy": [accuracy_score(true_train, pred_train), accuracy_score(true_val, pred_val)],
    "Precision": [precision_score(true_train, pred_train, average='macro'),
                  precision_score(true_val, pred_val, average='macro')],
    "Recall": [recall_score(true_train, pred_train, average='macro'),
               recall_score(true_val, pred_val, average='macro')],
    "F1": [f1_score(true_train, pred_train, average='macro'), f1_score(true_val, pred_val, average='macro')],
    "AUC": [auc_train, auc_val],
    "Kappa": [cohen_kappa_score(true_train, pred_train), cohen_kappa_score(true_val, pred_val)]
    }
# write them as data frame to save to CSV
import pandas as pd

metrics_df = pd.DataFrame(metrics, index=["Train", "Validation"])
metrics_df

,Accuracy,Precision,Recall,F1,AUC,Kappa
Train,0.93899,0.943789,0.939241,0.939891,0.999434,0.938681
Validation,0.93302,0.937528,0.933534,0.932861,0.999396,0.932676


In [33]:
# print Roc curve average ovr
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(200):
    fpr[i], tpr[i], _ = roc_curve((true_val == i).astype(int), prob_val[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area

fpr["micro"], tpr["micro"], _ = roc_curve((true_val == i).astype(int), prob_val.ravel())

roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# First aggregate all false positive rates

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(200)]))

# Then interpolate all ROC curves at this points

mean_tpr = np.zeros_like(all_fpr)
for i in range(200):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC

mean_tpr /= 200

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot overall ROC curve
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label=f'micro-average ROC curve (area = {roc_auc["micro"]:0.2f})',
         color='deeppink', linestyle=':', linewidth=4)





ValueError: Found input variables with inconsistent numbers of samples: [7450, 1490000]

In [23]:
true_val

array([  4,  31,  12, ...,  12, 195,  50])

In [15]:
metrics_df

,Accuracy,Precision,Recall,F1,AUC,Kappa
Train,0.93899,0.943789,0.939241,0.939891,0.999434,0.938681
Validation,0.93302,0.937528,0.933534,0.932861,0.999396,0.932676


In [60]:
# save the metrics to a CSV file
metrics_df.to_csv(f"{PATH}/metrics.csv")

In [ ]:
#TODO: investigate potential data leakage: shape of train (49,664) shape of val (7,450)

In [63]:
model_set.keys()

dict_keys(['model', 'epoch', 'batch_size', 'model_state_dict', 'optimizer_state_dict', 'train_loss', 'val_loss', 'train_acc', 'val_acc', 'all_train_loss', 'all_val_loss', 'all_train_acc', 'all_val_acc', 'train_loader', 'val_loader'])

In [66]:
model_set["all_train_acc"]

[0.004731797680412371,
 0.006986952319587629,
 0.03241784793814433,
 0.09258215206185567,
 0.1643041237113402,
 0.23129429768041238,
 0.28322326030927836,
 0.32429929123711343,
 0.35669700386597936,
 0.38144329896907214,
 0.40192090850515466,
 0.41925740979381443,
 0.4314191365979381,
 0.44833279639175255,
 0.45570231958762886,
 0.46498469716494845,
 0.47388450386597936,
 0.4829655283505155,
 0.48691204896907214,
 0.4951071198453608,
 0.4997986469072165,
 0.5040270618556701,
 0.5093830541237113,
 0.5194708440721649,
 0.5181419136597938,
 0.523779800257732,
 0.5344515141752577,
 0.5336461018041238,
 0.5368878865979382,
 0.5427875322164949,
 0.5431096971649485,
 0.551868556701031,
 0.5512242268041238,
 0.556237918814433,
 0.5553519652061856,
 0.5606475515463918,
 0.5581105025773195,
 0.5656209729381443,
 0.5660438144329897,
 0.5676546391752577,
 0.5708964239690721,
 0.5716414304123711,
 0.5720642719072165,
 0.5788297358247423,
 0.5802593427835051,
 0.5802593427835051,
 0.5784471649484536